# 1. Get the packages

In [590]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm

# 2. Get the Data

In [591]:
from cgi import test


train_df = pd.read_json('../data/entity/relations_training.txt')
test_df = pd.read_json('../data/entity/relations_test.txt') 


In [592]:
from cProfile import label


def pre_process_tokens(tokens):
    enti={}
    for t in tokens:
        
        if t['entityLabel'] in enti.keys():
            enti[t['entityLabel']] = enti[t['entityLabel']] +','+t['text']
        else:
            enti[t['entityLabel']] = t['text']
    label = ""
    for k, v in enti.items():
        label += k+':'+v+"\n"
    return label

In [593]:
train_df['label'] = train_df['tokens'].apply(pre_process_tokens)
test_df['label'] = test_df['tokens'].apply(pre_process_tokens)


In [594]:
test_df

,document,tokens,relations,label
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:machine learning,..."
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...",EXPERIENCE:2+ years\nSKILLS:online advertising...
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","EXPERIENCE:5+ years,2+ years\nSKILLS:project m..."
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","DIPLOMA:MS,PhD\nDIPLOMA_MAJOR:Operations Resea..."
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",DIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Computer Science\n
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","EXPERIENCE:12+ years\nSKILLS:research,Ph.D\nDI..."
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","EXPERIENCE:5+ years,3+ years,3+ years,5+ years..."


# 3. preprocess the data

In [595]:

train_doc = []
for i in range(train_df.shape[0]):
    ent = train_df.label.iloc[i]
    docu = train_df.document.iloc[i].replace("\n"," ")
    if len(ent)!=0:
        train_doc.append(docu+"\n\nExtracted Text:" +
                        '\n'+ent+"----\n")
    
with open('../data/output/entity.txt','w') as f:
    for item in train_doc:
        # write each item on a new line
        f.write("%s\n" % item.strip())
    print('Done')
    

Done


In [596]:

test_doc = test_df.document.apply(
    lambda x: x.replace("\n", " ")+'\n\nExtracted Text:').to_list()


In [597]:
import cohere
api_key = pd.read_json("../config/apikey.json")['cohere_key'][0]
co = cohere.Client(api_key)


# 4. Running the model

In [598]:
d ='dfsdffs'
d.split(":")

['dfsdffs']

In [599]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    def __init__(self, examples):
        self.examples = examples

    def make_prompt(self, example):
        prompt = self.examples + [example]
       
        return ("".join([str(exam) for exam in prompt]))

    def extract(self, example):
        extraction = co.generate(
            model='large',
            prompt=self.make_prompt(example),
            max_tokens=100,
            temperature=0.5,
            stop_sequences=["----"])
        
        return(extraction.generations[0].text[:-1])


cohereMovieExtractor = cohereExtractor(train_doc[:6])



In [600]:
print(cohereMovieExtractor.make_prompt(test_doc[-1]))


MS/Ph.D. in CS/EE or related areas is required. Strong ability and effectiveness working in a significant technical problem domain, in the term of plan, design, execution, continuous release and service operation. Strong software engineering fundamentals, including coding, problem solving and data analysis skills. Background in machine learning/deep learning (strongly preferred). Passionate and self-motivated. Ability to effectively work in collaborative multiple project team environment and ship production features in a fast-paced environment. Good communication skills, both verbal and written. Customer/end result/Metrics driven in design and development. Strong ability in self-learning, entering new domain, managing through uncertainty in an innovative team environment

Extracted Text:
DIPLOMA:MS/Ph.D.
DIPLOMA_MAJOR:CS/EE
----
1+ years development experience on Java stack AppConnect / API's experience is added advantage. Compute, Network and Storage Monitoring Tools (Ex: Netcool) App

In [601]:
results = []
for text in tqdm(test_doc):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        print(extracted_text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)


  9%|▉         | 1/11 [00:04<00:40,  4.09s/it]


DIPLOMA:PhD
DIPLOMA_MAJOR:Artificial Intelligence
---


 18%|█▊        | 2/11 [00:08<00:36,  4.03s/it]


EXPERIENCE:2+ years
SKILLS:research,online advertising,research
DIPLOMA:undergraduate
DIPLOMA_MAJOR:economics,statistics,political science,history,psychology
---


 27%|██▋       | 3/11 [00:11<00:31,  3.91s/it]

...


EXPERIENCE:12+ years
SKILLS:research,user safety,computer vision,human-computer interaction,human factors,cognitive psychology
DIPLOMA:Ph.D
DIPLOMA_MAJOR:Human Factors,Human Computer Interaction,Human Factors
---


 91%|█████████ | 10/11 [00:42<00:04,  4.37s/it]


EXPERIENCE:3+ years
SKILLS:management,compensation,performance management,diversity,compensation,retention
---


100%|██████████| 11/11 [00:46<00:00,  4.22s/it]


DIPLOMA:PhD
DIPLOMA_MAJOR:AI,Machine Learning,Statistics,Computational
---


In [602]:
pd.DataFrame(data={'text': test_doc, 'extracted_text': results})


,text,extracted_text
0,"Currently holding a faculty, industry, or gov...",\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Artificial Intell...
1,2+ years experience in the online advertising...,"\nEXPERIENCE:2+ years\nSKILLS:research,online ..."
2,BA/BS 5+ years of program or project manageme...,"\nEXPERIENCE:5+ years\nSKILLS:program,project\..."
3,Currently enrolled in a full-time degree prog...,\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Computer Science\...
4,Currently enrolled in a full-time degree prog...,\nDIPLOMA:PhD\nDIPLOMA_MAJOR:Computer Science\...
5,MS in a quantitative field such as Operations...,"\nEXPERIENCE:7+ years\nSKILLS:modeling,algorit..."
6,"Ph.D. with 5+ years of experience, MS with 7+...",\nDIPLOMA:Ph.D.\nDIPLOMA_MAJOR:Physics\n---
7,Graduating with a Ph.D. in Computer Science o...,\nDIPLOMA:Ph.D\nDIPLOMA_MAJOR:Computer Science...
8,"12+ years of research experience, in-house or...","\nEXPERIENCE:12+ years\nSKILLS:research,user s..."
9,"PhD with 3+ years, or MS with 5+ years of exp...","\nEXPERIENCE:3+ years\nSKILLS:management,compe..."


In [603]:
def clean_test(x):
    x = "".join(x)
    x = x.lower()
    x = x.lstrip()
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
    return x

def clean_extracted(x):
    x = "".join(x)
    x = x.lower()
    x = x[:-3]
    x = x.lstrip()
    
    x = x.split("\n")
    x = "".join(x)
    x = x.split(':')
    x = "".join(x)
   
    return x


In [604]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['document'].apply(
            lambda x: "Job: "+x.replace("\n", " ")+'\n\nExtracted Text:').to_list()):
        extracted.append(str(i).strip())



In [605]:
# Save results
test_df['extracted_text'] = extracted


In [606]:
test_df['extracted_text'] = test_df['extracted_text'].apply(clean_extracted)
test_df['label'] = test_df['label'].apply(clean_test)


In [607]:
from difflib import SequenceMatcher
score = []
for i in range(test_df.shape[0]):
    
    score.append(SequenceMatcher(None,test_df['label'].iloc[i],
                          test_df['extracted_text'].iloc[i]).ratio())

In [613]:
# Compare the label to the extracted text
test_df['similarity_score'] = pd.DataFrame(score)

# Print the accuracy
print(f'Classification accuracy {test_df["similarity_score"].mean() *100}%')


Classification accuracy 53.1272735803995%


In [609]:
test_df

,document,tokens,relations,label,extracted_text,similarity_score
0,"\nCurrently holding a faculty, industry, or go...","[{'text': 'Ph.D.', 'start': 75, 'end': 80, 'to...","[{'child': 18, 'head': 14, 'relationLabel': 'D...","diplomaph.d.diploma_majormachine learning,ai,c...",diplomaphddiploma_majorartificial intelligence,0.314815
1,\n2+ years experience in the online advertisin...,"[{'text': '2+ years', 'start': 1, 'end': 9, 't...","[{'child': 7, 'head': 1, 'relationLabel': 'EXP...","experience2+ yearsskillsonline advertising,res...","diplomamba,ms/ph.d.diploma_majorpolitical scie...",0.514056
2,\nBA/BS\n5+ years of program or project manage...,"[{'text': '5+ years', 'start': 7, 'end': 15, '...","[{'child': 11, 'head': 5, 'relationLabel': 'EX...","experience5+ years,2+ yearsskillsproject manag...","diplomaba/bsdiploma_majorengineering,project m...",0.335664
3,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diplomaph.d.diploma_majorcomputer science,diplomaphddiploma_majorcomputer science,0.975000
4,\nCurrently enrolled in a full-time degree pro...,"[{'text': 'Ph.D.', 'start': 801, 'end': 806, '...","[{'child': 140, 'head': 137, 'relationLabel': ...",diplomaph.d.diploma_majorcomputer science,diplomaphddiploma_majorcomputer science,0.975000
5,\nMS in a quantitative field such as Operation...,"[{'text': 'MS', 'start': 1, 'end': 3, 'token_s...","[{'child': 8, 'head': 1, 'relationLabel': 'DEG...","diplomams,phddiploma_majoroperations research,...","diplomamsdiploma_majoroperations research,comp...",0.380952
6,"\nPh.D. with 5+ years of experience, MS with 7...",[],[],,"diplomaph.d.,ms,bsdiploma_majorphysics,electri...",0.000000
7,\nGraduating with a Ph.D. in Computer Science ...,"[{'text': 'Ph.D.', 'start': 19, 'end': 24, 'to...","[{'child': 6, 'head': 4, 'relationLabel': 'DEG...",diplomaph.d.diploma_majorcomputer science,diplomaph.ddiploma_majorcomputer science,0.987654
8,"\n12+ years of research experience, in-house o...","[{'text': '12+ years', 'start': 1, 'end': 10, ...","[{'child': 5, 'head': 1, 'relationLabel': 'EXP...","experience12+ yearsskillsresearch,ph.ddiploma_...",diplomaph.d.diploma_majorhuman-computer intera...,0.808081
9,"\nPhD with 3+ years, or MS with 5+ years of ex...","[{'text': '5+ years', 'start': 31, 'end': 39, ...","[{'child': 18, 'head': 10, 'relationLabel': 'E...","experience5+ years,3+ years,3+ years,5+ yearss...","experience3+ yearsskillspeople research,organi...",0.247899


In [610]:
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# print(classification_report(test_df['label'], test_df['extracted_text']))
